In [33]:
import os
import pandas as pd
import shutil

In [34]:
# 파일 이름에서 값을 추출하는 함수
def parse_filename(file_path):
    file_name = os.path.basename(file_path)
    parts = file_name.split('_')
    return {
        'N': float(parts[0]),
        'L1': float(parts[1]),
        'L2': float(parts[2]),
        'W': float(parts[3]),
        'S': float(parts[4].replace('.csv', ''))
    }

In [35]:
# 음수 값을 포함한 주파수 이상의 모든 데이터를 제거하는 함수
def remove_negative_data(data):
    if 'phase(S(1,1))' in data.columns:
        freqs_to_remove = data[(data['phase(S(1,1))'] < 0) | (data['L'] < 0) | (data['Q'] < 0)]['freq'].unique()
    else:
        freqs_to_remove = data[(data['L'] < 0) | (data['Q'] < 0)]['freq'].unique()
    if freqs_to_remove.size > 0:
        min_freq_to_remove = min(freqs_to_remove)
        data = data[data['freq'] < min_freq_to_remove]
    return data


In [36]:
# 데이터 파일들이 있는 폴더 경로
folder_path = r'C:\\Users\\COMS03\\Desktop\\Capstone\\DataSet'
output_folder_path = r'C:\\Users\\COMS03\\Desktop\\Capstone\\Preprocessing DataSet'
success_folder_path = r'C:\\Users\\COMS03\\Desktop\\Capstone\\DataSet\\Success'
fail_folder_path = r'C:\\Users\\COMS03\\Desktop\\Capstone\\DataSet\\Fail'

# 출력 폴더가 존재하지 않으면 생성
os.makedirs(output_folder_path, exist_ok=True)
os.makedirs(success_folder_path, exist_ok=True)
os.makedirs(fail_folder_path, exist_ok=True)

In [37]:
# 폴더 내의 모든 파일을 읽어옴
all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# 각 파일의 데이터를 처리하고 저장
for file_path in all_files:
    # 파일 이름에서 값을 추출
    file_values = parse_filename(file_path)
    # 파일에서 데이터 불러오기
    data = pd.read_csv(file_path)
    # 추출된 값을 데이터프레임에 새로운 컬럼으로 추가
    for key, value in file_values.items():
        data[key] = value
    # 명확성을 위해 컬럼 이름 변경
    num_cols = data.shape[1]
    success = False
    if num_cols == 10:
        data.columns = ['freq', 'mag(S(1,1))', 'phase(S(1,1))', 'L', 'Q', 'N', 'L1', 'L2', 'W', 'S']
        success = True
    elif num_cols == 8:
        data.columns = ['freq', 'L', 'Q', 'N', 'L1', 'L2', 'W', 'S']
        success = True
    else:
        print(f'Unexpected number of columns in file {file_path}: {num_cols}')

    if success:
        # 음수 값을 포함한 주파수 이상의 데이터 제거
        data = remove_negative_data(data)
        # 처리된 데이터를 새로운 파일로 저장
        output_file_path = os.path.join(output_folder_path, os.path.basename(file_path))
        data.to_csv(output_file_path, index=False)
        print(f'Processed and saved: {output_file_path}')
        # 성공 폴더로 이동
        shutil.move(file_path, os.path.join(success_folder_path, os.path.basename(file_path)))
    else:
        # 실패 폴더로 이동
        shutil.move(file_path, os.path.join(fail_folder_path, os.path.basename(file_path)))


Processed and saved: C:\\Users\\COMS03\\Desktop\\Capstone\\Preprocessing DataSet\1.00_2.50_2.50_0.10_0.10.csv
Processed and saved: C:\\Users\\COMS03\\Desktop\\Capstone\\Preprocessing DataSet\1.00_2.50_3.00_0.10_0.10.csv
Processed and saved: C:\\Users\\COMS03\\Desktop\\Capstone\\Preprocessing DataSet\1.00_2.50_3.50_0.10_0.10.csv
Processed and saved: C:\\Users\\COMS03\\Desktop\\Capstone\\Preprocessing DataSet\1.00_2.50_4.00_0.10_0.10.csv
Processed and saved: C:\\Users\\COMS03\\Desktop\\Capstone\\Preprocessing DataSet\1.00_3.00_2.50_0.10_0.10.csv
Processed and saved: C:\\Users\\COMS03\\Desktop\\Capstone\\Preprocessing DataSet\1.00_3.00_3.00_0.10_0.10.csv
Processed and saved: C:\\Users\\COMS03\\Desktop\\Capstone\\Preprocessing DataSet\1.00_3.00_3.50_0.10_0.10.csv
Processed and saved: C:\\Users\\COMS03\\Desktop\\Capstone\\Preprocessing DataSet\1.00_3.00_4.00_0.10_0.10.csv
Processed and saved: C:\\Users\\COMS03\\Desktop\\Capstone\\Preprocessing DataSet\1.00_3.50_2.50_0.10_0.10.csv
Processed 